В данном ноутбуке представлен подход к обучению доп модели для малочисленных классов.

Из предыдущих опытов стало ясно, что малочисленные классы предсказываются довольно плохо. Как один из подходов к улучшению предсказаний малочисленных классов был выбран подход с обучением несколько моделей: одна модель обучается на малочисленных классах, другая на либо на всех оставшихся, либо вообще на всех класс, включая малочисленные. Далее будут сравниваться уверенности классификатрах (метод predict_proba) на конкретном объекте и выбираться для него тот класс наиболее уверенного классификатора.

In [68]:
#!pip install catboost

In [1]:
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
from catboost import Pool
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import time

In [5]:
# from google.colab import drive
# drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
#df = pd.read_csv('/content/drive/MyDrive/data/lenta_mystem_lemmas_str_largecol_v02.csv')
df = pd.read_csv('lenta_mystem_lemmas_str_largecol_v02.csv')

In [3]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df

,date,url,topic,tags,text_str,topic_le,title_lemmas,year,month,day,date_enc,season,day_of_week,dummy_weekday
0,1999-08-31,https://lenta.ru/news/1999/08/31/stancia_mir/,Россия,Все,становиться известно агентство ассошиэйтед пре...,20,космонавт сомневаться надежность мир,1999,8,31,0,3,1,1
1,1999-08-31,https://lenta.ru/news/1999/08/31/vzriv/,Россия,Все,зал игровой автомат третий ярус подземный комп...,20,взрыв центр москва пострадать человек,1999,8,31,0,3,1,1
2,1999-08-31,https://lenta.ru/news/1999/08/31/credit_japs/,Россия,Все,япония принимать решение размораживать кредит ...,20,япония кредитовать россия полтора миллиард доллар,1999,8,31,0,3,1,1
3,1999-08-31,https://lenta.ru/news/1999/08/31/diana/,Мир,Все,британец отмечать сегодня скорбный дата вторую...,13,британец отмечать двухлетие смерть диана,1999,8,31,0,3,1,1
4,1999-08-31,https://lenta.ru/news/1999/08/31/mvf/,Россия,Все,понедельник директор департамент внешний связь...,20,отмывать bank деньги иметь отношение мвф,1999,8,31,0,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1153089,2022-11-18,https://lenta.ru/news/2022/11/18/bezhh/,Мир,Общество,беженец украина избивать город кейзер американ...,13,беженец украина избивать спор нацист,2022,11,18,8472,4,4,1
1153090,2022-11-18,https://lenta.ru/news/2022/11/18/nurmagomedov/,Спорт,Бокс и ММА,российский боец смешанный стиль мма двоюродный...,23,брат нурмагомедов называть цель выступление be...,2022,11,18,8472,4,4,1
1153091,2022-11-18,https://lenta.ru/news/2022/11/18/bioprint/,Наука и техника,Будущее,ученый израильский технологический институт ра...,16,разрабатывать технология печать ткань транспла...,2022,11,18,8472,4,4,1
1153092,2022-11-18,https://lenta.ru/news/2022/11/18/penni/,Мир,Политика,республиканец палата представитель конгресс сш...,13,республиканец конгресс потребовать проверять к...,2022,11,18,8472,4,4,1


In [4]:
df.columns

Index(['date', 'url', 'topic', 'tags', 'text_str', 'topic_le', 'title_lemmas',
       'year', 'month', 'day', 'date_enc', 'season', 'day_of_week',
       'dummy_weekday'],
      dtype='object')

In [5]:
df['topic'].value_counts().index[-6:]

Index(['Библиотека', 'Оружие', 'ЧМ-2014', 'Мотор', 'МедНовости', 'Сочи'], dtype='object')

In [6]:
names = list(df['topic'].value_counts().index[-6:])
df = df[~df['topic'].isin(names)]
df['topic'].value_counts()

Россия               241055
Мир                  195778
Экономика            122120
Бывший СССР           93769
Спорт                 91267
Наука и техника       75609
Культура              71765
Интернет и СМИ        64800
Из жизни              44812
Силовые структуры     43287
Путешествия           22756
Ценности              22323
Дом                   22132
Среда обитания        15607
Бизнес                 7430
Моя страна             6128
Нацпроекты             4728
69-я параллель         3356
Забота о себе          3174
Крым                    666
Культпросвет            340
Легпром                 114
Name: topic, dtype: int64

In [7]:
dftrain, dftest, ytrain, ytest = train_test_split(df, df['topic_le'],
                                                  test_size=0.1,
                                                  stratify=df['topic'], 
                                                  random_state=123)

Xtrain, Xtest, ytrain, ytest = train_test_split(dftrain, dftrain['topic_le'],
                                                  test_size=0.2,
                                                  stratify=dftrain['topic'],
                                                  random_state=123)

X1, X, y1, y = train_test_split(Xtrain, Xtrain['topic_le'],
                                                  test_size=0.05,
                                                  stratify=Xtrain['topic'],
                                                  random_state=123)

In [8]:
del df
del dftest
del X1

In [9]:
Xtest.shape

(207543, 14)

Разобьем тестовый датасет на 2:
 - `X_test_often` - датасет с темами, которые встречаются часто
 - `Xtest_rarely` - датасет с темами, которые встречаются часто (туда вошли 3 темы: Крым, Культпросвет и Легпром). Темы отбирались на основании метрик качества предсказаний, полученных с помощью CatBoost, и кол-ву наблюдений в ней. Если метрика f1-score была низкой (меньше 0.6), то такая тема включалась в список малочисленных классов

In [10]:
Xtest['y'] = ytest
names = list(Xtest['topic'].value_counts().index[-3:])
Xtest_often = Xtest[~Xtest['topic'].isin(names)]
Xtest_rarely = Xtest[Xtest['topic'].isin(names)]

In [11]:
# Соотнесения закодированного таргета и названий категорий
dict_topic = dict(zip(Xtest.topic, Xtest.topic_le))
dict_topic = dict(sorted(dict_topic.items(), key=lambda item: item[1]))

dict_topic_often = dict(zip(Xtest_often.topic, Xtest_often.topic_le))
dict_topic_often = dict(sorted(dict_topic_often.items(), key=lambda item: item[1]))

dict_topic_rarely = dict(zip(Xtest_rarely.topic, Xtest_rarely.topic_le))
dict_topic_rarely = dict(sorted(dict_topic_rarely.items(), key=lambda item: item[1]))

In [59]:
X_test_often = Xtest_often[['text_str', 'title_lemmas', 'date_enc', 'year', 'season', 'day_of_week']]
X_test_rarely = Xtest_rarely[['text_str', 'title_lemmas', 'date_enc', 'year', 'season', 'day_of_week']]
X_test = Xtest[['text_str', 'title_lemmas', 'date_enc', 'year', 'season', 'day_of_week']]

text_cols = ['text_str', 'title_lemmas']
cat_cols = ['year', 'day_of_week', 'season']

In [58]:
clf = CatBoostClassifier(random_state=123, 
                         loss_function='MultiClass', 
                         eval_metric='TotalF1', 
                         task_type='GPU',
                         n_estimators=1000)

In [13]:
clf1 = CatBoostClassifier(random_state=123, 
                         loss_function='MultiClass', 
                         eval_metric='TotalF1', 
                         task_type='GPU',
                         n_estimators=1000)

In [14]:
clf2 = CatBoostClassifier(random_state=123, 
                         loss_function='MultiClass', 
                         eval_metric='TotalF1', 
                         task_type='GPU',
                         n_estimators=1000)

Обучим Catboost с дефолтными настройками на всем тестовой датасете.

In [60]:
clf.fit(X_test, ytest, text_features=text_cols, cat_features=cat_cols)

Learning rate set to 0.192741
0:	learn: 0.5788394	total: 531ms	remaining: 8m 50s
1:	learn: 0.6620928	total: 915ms	remaining: 7m 36s
2:	learn: 0.7142629	total: 1.26s	remaining: 6m 59s
3:	learn: 0.7359953	total: 1.68s	remaining: 6m 57s
4:	learn: 0.7473362	total: 2.1s	remaining: 6m 58s
5:	learn: 0.7497188	total: 2.49s	remaining: 6m 52s
6:	learn: 0.7514222	total: 2.96s	remaining: 6m 59s
7:	learn: 0.7542792	total: 3.41s	remaining: 7m 2s
8:	learn: 0.7572534	total: 3.78s	remaining: 6m 56s
9:	learn: 0.7624432	total: 4.15s	remaining: 6m 50s
10:	learn: 0.7638151	total: 4.54s	remaining: 6m 48s
11:	learn: 0.7660808	total: 5s	remaining: 6m 51s
12:	learn: 0.7676354	total: 5.44s	remaining: 6m 53s
13:	learn: 0.7688212	total: 5.83s	remaining: 6m 50s
14:	learn: 0.7727027	total: 6.19s	remaining: 6m 46s
15:	learn: 0.7744147	total: 6.59s	remaining: 6m 45s
16:	learn: 0.7749470	total: 6.93s	remaining: 6m 41s
17:	learn: 0.7766689	total: 7.39s	remaining: 6m 42s
18:	learn: 0.7773939	total: 7.81s	remaining: 6m 4

158:	learn: 0.8336528	total: 60s	remaining: 5m 17s
159:	learn: 0.8337874	total: 1m	remaining: 5m 16s
160:	learn: 0.8340240	total: 1m	remaining: 5m 16s
161:	learn: 0.8341212	total: 1m 1s	remaining: 5m 16s
162:	learn: 0.8343160	total: 1m 1s	remaining: 5m 15s
163:	learn: 0.8345020	total: 1m 1s	remaining: 5m 15s
164:	learn: 0.8346171	total: 1m 2s	remaining: 5m 14s
165:	learn: 0.8347833	total: 1m 2s	remaining: 5m 14s
166:	learn: 0.8348836	total: 1m 2s	remaining: 5m 13s
167:	learn: 0.8351458	total: 1m 3s	remaining: 5m 13s
168:	learn: 0.8353260	total: 1m 3s	remaining: 5m 12s
169:	learn: 0.8355866	total: 1m 4s	remaining: 5m 12s
170:	learn: 0.8358060	total: 1m 4s	remaining: 5m 12s
171:	learn: 0.8359761	total: 1m 4s	remaining: 5m 12s
172:	learn: 0.8361800	total: 1m 5s	remaining: 5m 11s
173:	learn: 0.8363232	total: 1m 5s	remaining: 5m 11s
174:	learn: 0.8364099	total: 1m 5s	remaining: 5m 11s
175:	learn: 0.8366864	total: 1m 6s	remaining: 5m 10s
176:	learn: 0.8368287	total: 1m 6s	remaining: 5m 10s
1

311:	learn: 0.8527539	total: 1m 54s	remaining: 4m 12s
312:	learn: 0.8527922	total: 1m 54s	remaining: 4m 12s
313:	learn: 0.8528391	total: 1m 55s	remaining: 4m 11s
314:	learn: 0.8528615	total: 1m 55s	remaining: 4m 11s
315:	learn: 0.8529644	total: 1m 56s	remaining: 4m 11s
316:	learn: 0.8531118	total: 1m 56s	remaining: 4m 10s
317:	learn: 0.8532537	total: 1m 56s	remaining: 4m 10s
318:	learn: 0.8531837	total: 1m 57s	remaining: 4m 10s
319:	learn: 0.8533309	total: 1m 57s	remaining: 4m 9s
320:	learn: 0.8534793	total: 1m 57s	remaining: 4m 9s
321:	learn: 0.8535130	total: 1m 58s	remaining: 4m 8s
322:	learn: 0.8536652	total: 1m 58s	remaining: 4m 8s
323:	learn: 0.8537344	total: 1m 58s	remaining: 4m 7s
324:	learn: 0.8537657	total: 1m 59s	remaining: 4m 7s
325:	learn: 0.8539795	total: 1m 59s	remaining: 4m 7s
326:	learn: 0.8540420	total: 1m 59s	remaining: 4m 6s
327:	learn: 0.8540608	total: 2m	remaining: 4m 6s
328:	learn: 0.8542227	total: 2m	remaining: 4m 5s
329:	learn: 0.8543150	total: 2m	remaining: 4m 

465:	learn: 0.8639288	total: 2m 46s	remaining: 3m 11s
466:	learn: 0.8639715	total: 2m 47s	remaining: 3m 10s
467:	learn: 0.8640955	total: 2m 47s	remaining: 3m 10s
468:	learn: 0.8640870	total: 2m 47s	remaining: 3m 10s
469:	learn: 0.8641495	total: 2m 48s	remaining: 3m 9s
470:	learn: 0.8641794	total: 2m 48s	remaining: 3m 9s
471:	learn: 0.8642914	total: 2m 48s	remaining: 3m 8s
472:	learn: 0.8642981	total: 2m 49s	remaining: 3m 8s
473:	learn: 0.8643361	total: 2m 49s	remaining: 3m 8s
474:	learn: 0.8644021	total: 2m 49s	remaining: 3m 7s
475:	learn: 0.8644758	total: 2m 50s	remaining: 3m 7s
476:	learn: 0.8645361	total: 2m 50s	remaining: 3m 7s
477:	learn: 0.8645913	total: 2m 50s	remaining: 3m 6s
478:	learn: 0.8646703	total: 2m 51s	remaining: 3m 6s
479:	learn: 0.8647668	total: 2m 51s	remaining: 3m 5s
480:	learn: 0.8648033	total: 2m 52s	remaining: 3m 5s
481:	learn: 0.8648906	total: 2m 52s	remaining: 3m 5s
482:	learn: 0.8649552	total: 2m 52s	remaining: 3m 4s
483:	learn: 0.8649288	total: 2m 53s	remain

619:	learn: 0.8717086	total: 3m 37s	remaining: 2m 13s
620:	learn: 0.8717009	total: 3m 38s	remaining: 2m 13s
621:	learn: 0.8717369	total: 3m 38s	remaining: 2m 12s
622:	learn: 0.8718078	total: 3m 38s	remaining: 2m 12s
623:	learn: 0.8718212	total: 3m 39s	remaining: 2m 12s
624:	learn: 0.8718856	total: 3m 39s	remaining: 2m 11s
625:	learn: 0.8719051	total: 3m 39s	remaining: 2m 11s
626:	learn: 0.8719152	total: 3m 40s	remaining: 2m 10s
627:	learn: 0.8720543	total: 3m 40s	remaining: 2m 10s
628:	learn: 0.8721521	total: 3m 40s	remaining: 2m 10s
629:	learn: 0.8721729	total: 3m 41s	remaining: 2m 9s
630:	learn: 0.8721677	total: 3m 41s	remaining: 2m 9s
631:	learn: 0.8721728	total: 3m 41s	remaining: 2m 9s
632:	learn: 0.8721823	total: 3m 42s	remaining: 2m 8s
633:	learn: 0.8723044	total: 3m 42s	remaining: 2m 8s
634:	learn: 0.8722846	total: 3m 42s	remaining: 2m 8s
635:	learn: 0.8723592	total: 3m 43s	remaining: 2m 7s
636:	learn: 0.8723908	total: 3m 43s	remaining: 2m 7s
637:	learn: 0.8724045	total: 3m 43s	

773:	learn: 0.8778963	total: 4m 27s	remaining: 1m 18s
774:	learn: 0.8779068	total: 4m 27s	remaining: 1m 17s
775:	learn: 0.8778996	total: 4m 28s	remaining: 1m 17s
776:	learn: 0.8779047	total: 4m 28s	remaining: 1m 17s
777:	learn: 0.8779092	total: 4m 28s	remaining: 1m 16s
778:	learn: 0.8779399	total: 4m 29s	remaining: 1m 16s
779:	learn: 0.8780806	total: 4m 29s	remaining: 1m 15s
780:	learn: 0.8781765	total: 4m 29s	remaining: 1m 15s
781:	learn: 0.8782001	total: 4m 30s	remaining: 1m 15s
782:	learn: 0.8782292	total: 4m 30s	remaining: 1m 14s
783:	learn: 0.8782493	total: 4m 30s	remaining: 1m 14s
784:	learn: 0.8783309	total: 4m 31s	remaining: 1m 14s
785:	learn: 0.8783405	total: 4m 31s	remaining: 1m 13s
786:	learn: 0.8783824	total: 4m 31s	remaining: 1m 13s
787:	learn: 0.8784118	total: 4m 31s	remaining: 1m 13s
788:	learn: 0.8784505	total: 4m 32s	remaining: 1m 12s
789:	learn: 0.8785367	total: 4m 32s	remaining: 1m 12s
790:	learn: 0.8786003	total: 4m 32s	remaining: 1m 12s
791:	learn: 0.8786609	total:

929:	learn: 0.8830248	total: 5m 17s	remaining: 23.9s
930:	learn: 0.8830789	total: 5m 17s	remaining: 23.5s
931:	learn: 0.8830746	total: 5m 17s	remaining: 23.2s
932:	learn: 0.8831023	total: 5m 17s	remaining: 22.8s
933:	learn: 0.8831899	total: 5m 18s	remaining: 22.5s
934:	learn: 0.8831808	total: 5m 18s	remaining: 22.1s
935:	learn: 0.8831704	total: 5m 18s	remaining: 21.8s
936:	learn: 0.8831944	total: 5m 19s	remaining: 21.5s
937:	learn: 0.8832216	total: 5m 19s	remaining: 21.1s
938:	learn: 0.8832511	total: 5m 19s	remaining: 20.8s
939:	learn: 0.8832670	total: 5m 20s	remaining: 20.4s
940:	learn: 0.8832995	total: 5m 20s	remaining: 20.1s
941:	learn: 0.8833590	total: 5m 20s	remaining: 19.7s
942:	learn: 0.8833940	total: 5m 20s	remaining: 19.4s
943:	learn: 0.8834224	total: 5m 21s	remaining: 19.1s
944:	learn: 0.8834213	total: 5m 21s	remaining: 18.7s
945:	learn: 0.8834532	total: 5m 21s	remaining: 18.4s
946:	learn: 0.8834943	total: 5m 22s	remaining: 18s
947:	learn: 0.8835504	total: 5m 22s	remaining: 1

Обучим Catboost с дефолтными настройками на датасете с темами, которые часто встречаются.

In [15]:
clf1.fit(X_test_often, Xtest_often['y'], text_features=text_cols, cat_features=cat_cols)

Learning rate set to 0.192703
0:	learn: 0.5807475	total: 460ms	remaining: 7m 39s
1:	learn: 0.6474680	total: 895ms	remaining: 7m 26s
2:	learn: 0.6957065	total: 1.23s	remaining: 6m 48s
3:	learn: 0.7316471	total: 1.52s	remaining: 6m 19s
4:	learn: 0.7342796	total: 1.86s	remaining: 6m 10s
5:	learn: 0.7359691	total: 2.23s	remaining: 6m 10s
6:	learn: 0.7502571	total: 2.53s	remaining: 5m 59s
7:	learn: 0.7530264	total: 2.93s	remaining: 6m 3s
8:	learn: 0.7558111	total: 3.28s	remaining: 6m 1s
9:	learn: 0.7596491	total: 3.6s	remaining: 5m 55s
10:	learn: 0.7606876	total: 3.95s	remaining: 5m 55s
11:	learn: 0.7653308	total: 4.3s	remaining: 5m 54s
12:	learn: 0.7664570	total: 4.6s	remaining: 5m 49s
13:	learn: 0.7675832	total: 4.97s	remaining: 5m 49s
14:	learn: 0.7693362	total: 5.36s	remaining: 5m 52s
15:	learn: 0.7715844	total: 5.7s	remaining: 5m 50s
16:	learn: 0.7731567	total: 6.05s	remaining: 5m 49s
17:	learn: 0.7768720	total: 6.31s	remaining: 5m 44s
18:	learn: 0.7776111	total: 6.63s	remaining: 5m 42

158:	learn: 0.8345918	total: 50.4s	remaining: 4m 26s
159:	learn: 0.8347476	total: 50.7s	remaining: 4m 25s
160:	learn: 0.8348734	total: 51s	remaining: 4m 25s
161:	learn: 0.8350079	total: 51.2s	remaining: 4m 24s
162:	learn: 0.8350729	total: 51.5s	remaining: 4m 24s
163:	learn: 0.8352546	total: 51.8s	remaining: 4m 23s
164:	learn: 0.8353918	total: 52.1s	remaining: 4m 23s
165:	learn: 0.8355627	total: 52.4s	remaining: 4m 23s
166:	learn: 0.8358603	total: 52.6s	remaining: 4m 22s
167:	learn: 0.8359949	total: 52.9s	remaining: 4m 22s
168:	learn: 0.8360001	total: 53.2s	remaining: 4m 21s
169:	learn: 0.8362140	total: 53.5s	remaining: 4m 21s
170:	learn: 0.8364804	total: 53.9s	remaining: 4m 21s
171:	learn: 0.8365549	total: 54.2s	remaining: 4m 21s
172:	learn: 0.8366137	total: 54.5s	remaining: 4m 20s
173:	learn: 0.8367863	total: 54.8s	remaining: 4m 20s
174:	learn: 0.8373141	total: 55.2s	remaining: 4m 20s
175:	learn: 0.8374055	total: 55.5s	remaining: 4m 19s
176:	learn: 0.8375371	total: 55.8s	remaining: 4m

312:	learn: 0.8544192	total: 1m 36s	remaining: 3m 31s
313:	learn: 0.8544348	total: 1m 36s	remaining: 3m 31s
314:	learn: 0.8545354	total: 1m 36s	remaining: 3m 30s
315:	learn: 0.8546963	total: 1m 37s	remaining: 3m 30s
316:	learn: 0.8548163	total: 1m 37s	remaining: 3m 30s
317:	learn: 0.8549899	total: 1m 37s	remaining: 3m 29s
318:	learn: 0.8550370	total: 1m 38s	remaining: 3m 29s
319:	learn: 0.8552522	total: 1m 38s	remaining: 3m 29s
320:	learn: 0.8554177	total: 1m 38s	remaining: 3m 28s
321:	learn: 0.8554617	total: 1m 39s	remaining: 3m 28s
322:	learn: 0.8554622	total: 1m 39s	remaining: 3m 28s
323:	learn: 0.8556185	total: 1m 39s	remaining: 3m 27s
324:	learn: 0.8557404	total: 1m 39s	remaining: 3m 27s
325:	learn: 0.8558370	total: 1m 40s	remaining: 3m 26s
326:	learn: 0.8560084	total: 1m 40s	remaining: 3m 26s
327:	learn: 0.8561277	total: 1m 40s	remaining: 3m 26s
328:	learn: 0.8561515	total: 1m 41s	remaining: 3m 25s
329:	learn: 0.8562361	total: 1m 41s	remaining: 3m 25s
330:	learn: 0.8561846	total:

466:	learn: 0.8653123	total: 2m 20s	remaining: 2m 40s
467:	learn: 0.8654157	total: 2m 20s	remaining: 2m 39s
468:	learn: 0.8654864	total: 2m 20s	remaining: 2m 39s
469:	learn: 0.8655401	total: 2m 21s	remaining: 2m 39s
470:	learn: 0.8655581	total: 2m 21s	remaining: 2m 38s
471:	learn: 0.8656589	total: 2m 21s	remaining: 2m 38s
472:	learn: 0.8658337	total: 2m 22s	remaining: 2m 38s
473:	learn: 0.8659580	total: 2m 22s	remaining: 2m 38s
474:	learn: 0.8659607	total: 2m 22s	remaining: 2m 37s
475:	learn: 0.8660422	total: 2m 23s	remaining: 2m 37s
476:	learn: 0.8660861	total: 2m 23s	remaining: 2m 37s
477:	learn: 0.8662563	total: 2m 23s	remaining: 2m 36s
478:	learn: 0.8662755	total: 2m 23s	remaining: 2m 36s
479:	learn: 0.8662992	total: 2m 24s	remaining: 2m 36s
480:	learn: 0.8663463	total: 2m 24s	remaining: 2m 35s
481:	learn: 0.8663307	total: 2m 24s	remaining: 2m 35s
482:	learn: 0.8662987	total: 2m 24s	remaining: 2m 35s
483:	learn: 0.8663529	total: 2m 25s	remaining: 2m 34s
484:	learn: 0.8663754	total:

619:	learn: 0.8727554	total: 3m 2s	remaining: 1m 51s
620:	learn: 0.8727513	total: 3m 2s	remaining: 1m 51s
621:	learn: 0.8727539	total: 3m 2s	remaining: 1m 50s
622:	learn: 0.8728330	total: 3m 2s	remaining: 1m 50s
623:	learn: 0.8728088	total: 3m 3s	remaining: 1m 50s
624:	learn: 0.8728773	total: 3m 3s	remaining: 1m 50s
625:	learn: 0.8729208	total: 3m 3s	remaining: 1m 49s
626:	learn: 0.8729117	total: 3m 4s	remaining: 1m 49s
627:	learn: 0.8728732	total: 3m 4s	remaining: 1m 49s
628:	learn: 0.8729477	total: 3m 4s	remaining: 1m 48s
629:	learn: 0.8729972	total: 3m 4s	remaining: 1m 48s
630:	learn: 0.8731425	total: 3m 5s	remaining: 1m 48s
631:	learn: 0.8731736	total: 3m 5s	remaining: 1m 47s
632:	learn: 0.8732276	total: 3m 5s	remaining: 1m 47s
633:	learn: 0.8733550	total: 3m 6s	remaining: 1m 47s
634:	learn: 0.8734075	total: 3m 6s	remaining: 1m 47s
635:	learn: 0.8734573	total: 3m 6s	remaining: 1m 46s
636:	learn: 0.8735159	total: 3m 6s	remaining: 1m 46s
637:	learn: 0.8735355	total: 3m 7s	remaining: 

772:	learn: 0.8787034	total: 3m 43s	remaining: 1m 5s
773:	learn: 0.8787078	total: 3m 43s	remaining: 1m 5s
774:	learn: 0.8787843	total: 3m 44s	remaining: 1m 5s
775:	learn: 0.8788517	total: 3m 44s	remaining: 1m 4s
776:	learn: 0.8788838	total: 3m 44s	remaining: 1m 4s
777:	learn: 0.8788961	total: 3m 44s	remaining: 1m 4s
778:	learn: 0.8789664	total: 3m 45s	remaining: 1m 3s
779:	learn: 0.8789612	total: 3m 45s	remaining: 1m 3s
780:	learn: 0.8790992	total: 3m 45s	remaining: 1m 3s
781:	learn: 0.8791096	total: 3m 45s	remaining: 1m 2s
782:	learn: 0.8790882	total: 3m 46s	remaining: 1m 2s
783:	learn: 0.8791893	total: 3m 46s	remaining: 1m 2s
784:	learn: 0.8792297	total: 3m 46s	remaining: 1m 2s
785:	learn: 0.8792394	total: 3m 46s	remaining: 1m 1s
786:	learn: 0.8792308	total: 3m 47s	remaining: 1m 1s
787:	learn: 0.8792154	total: 3m 47s	remaining: 1m 1s
788:	learn: 0.8792860	total: 3m 47s	remaining: 1m
789:	learn: 0.8793441	total: 3m 48s	remaining: 1m
790:	learn: 0.8794162	total: 3m 48s	remaining: 1m
79

929:	learn: 0.8839120	total: 4m 25s	remaining: 20s
930:	learn: 0.8839125	total: 4m 25s	remaining: 19.7s
931:	learn: 0.8839412	total: 4m 25s	remaining: 19.4s
932:	learn: 0.8839335	total: 4m 25s	remaining: 19.1s
933:	learn: 0.8839328	total: 4m 26s	remaining: 18.8s
934:	learn: 0.8839763	total: 4m 26s	remaining: 18.5s
935:	learn: 0.8840100	total: 4m 26s	remaining: 18.2s
936:	learn: 0.8840694	total: 4m 26s	remaining: 17.9s
937:	learn: 0.8841134	total: 4m 27s	remaining: 17.7s
938:	learn: 0.8841646	total: 4m 27s	remaining: 17.4s
939:	learn: 0.8842129	total: 4m 27s	remaining: 17.1s
940:	learn: 0.8842332	total: 4m 27s	remaining: 16.8s
941:	learn: 0.8842077	total: 4m 28s	remaining: 16.5s
942:	learn: 0.8843144	total: 4m 28s	remaining: 16.2s
943:	learn: 0.8842903	total: 4m 28s	remaining: 15.9s
944:	learn: 0.8843365	total: 4m 28s	remaining: 15.7s
945:	learn: 0.8843957	total: 4m 29s	remaining: 15.4s
946:	learn: 0.8844598	total: 4m 29s	remaining: 15.1s
947:	learn: 0.8844314	total: 4m 29s	remaining: 1

Обучим Catboost с дефолтными настройками на датасете с темами, которые редко встречаются.

In [16]:
clf2.fit(X_test_rarely, Xtest_rarely['y'], text_features=text_cols, cat_features=cat_cols)

Learning rate set to 0.046836
0:	learn: 0.9555832	total: 18.1ms	remaining: 18s
1:	learn: 0.9506345	total: 38.9ms	remaining: 19.4s
2:	learn: 0.9605851	total: 56.5ms	remaining: 18.8s
3:	learn: 0.9654532	total: 74.7ms	remaining: 18.6s
4:	learn: 0.9704244	total: 88.9ms	remaining: 17.7s
5:	learn: 0.9704244	total: 105ms	remaining: 17.4s
6:	learn: 0.9704244	total: 120ms	remaining: 17.1s
7:	learn: 0.9752385	total: 135ms	remaining: 16.7s
8:	learn: 0.9702571	total: 151ms	remaining: 16.7s
9:	learn: 0.9702571	total: 168ms	remaining: 16.6s
10:	learn: 0.9702571	total: 181ms	remaining: 16.3s
11:	learn: 0.9702571	total: 195ms	remaining: 16.1s
12:	learn: 0.9702571	total: 209ms	remaining: 15.9s
13:	learn: 0.9752385	total: 223ms	remaining: 15.7s
14:	learn: 0.9752385	total: 238ms	remaining: 15.6s
15:	learn: 0.9752385	total: 252ms	remaining: 15.5s
16:	learn: 0.9752385	total: 269ms	remaining: 15.6s
17:	learn: 0.9752385	total: 286ms	remaining: 15.6s
18:	learn: 0.9752385	total: 301ms	remaining: 15.6s
19:	lear

173:	learn: 0.9851180	total: 2.66s	remaining: 12.6s
174:	learn: 0.9851180	total: 2.68s	remaining: 12.6s
175:	learn: 0.9851180	total: 2.69s	remaining: 12.6s
176:	learn: 0.9851180	total: 2.71s	remaining: 12.6s
177:	learn: 0.9851180	total: 2.72s	remaining: 12.6s
178:	learn: 0.9851180	total: 2.74s	remaining: 12.6s
179:	learn: 0.9851180	total: 2.75s	remaining: 12.5s
180:	learn: 0.9851180	total: 2.77s	remaining: 12.5s
181:	learn: 0.9851180	total: 2.78s	remaining: 12.5s
182:	learn: 0.9851180	total: 2.8s	remaining: 12.5s
183:	learn: 0.9851180	total: 2.81s	remaining: 12.5s
184:	learn: 0.9851180	total: 2.83s	remaining: 12.5s
185:	learn: 0.9851180	total: 2.84s	remaining: 12.4s
186:	learn: 0.9851180	total: 2.86s	remaining: 12.4s
187:	learn: 0.9851180	total: 2.87s	remaining: 12.4s
188:	learn: 0.9851180	total: 2.88s	remaining: 12.4s
189:	learn: 0.9851180	total: 2.9s	remaining: 12.4s
190:	learn: 0.9851180	total: 2.92s	remaining: 12.4s
191:	learn: 0.9851180	total: 2.93s	remaining: 12.3s
192:	learn: 0.

336:	learn: 0.9900990	total: 5.1s	remaining: 10s
337:	learn: 0.9900990	total: 5.12s	remaining: 10s
338:	learn: 0.9900990	total: 5.13s	remaining: 10s
339:	learn: 0.9900990	total: 5.15s	remaining: 9.99s
340:	learn: 0.9900990	total: 5.16s	remaining: 9.97s
341:	learn: 0.9900990	total: 5.18s	remaining: 9.96s
342:	learn: 0.9900990	total: 5.19s	remaining: 9.95s
343:	learn: 0.9900990	total: 5.21s	remaining: 9.93s
344:	learn: 0.9900990	total: 5.22s	remaining: 9.92s
345:	learn: 0.9900990	total: 5.24s	remaining: 9.9s
346:	learn: 0.9900990	total: 5.25s	remaining: 9.89s
347:	learn: 0.9900990	total: 5.27s	remaining: 9.87s
348:	learn: 0.9900990	total: 5.29s	remaining: 9.86s
349:	learn: 0.9900990	total: 5.3s	remaining: 9.85s
350:	learn: 0.9900990	total: 5.32s	remaining: 9.84s
351:	learn: 0.9900990	total: 5.34s	remaining: 9.83s
352:	learn: 0.9900990	total: 5.35s	remaining: 9.81s
353:	learn: 0.9900990	total: 5.37s	remaining: 9.8s
354:	learn: 0.9900990	total: 5.38s	remaining: 9.78s
355:	learn: 0.9900990	

495:	learn: 0.9900990	total: 7.54s	remaining: 7.67s
496:	learn: 0.9900990	total: 7.56s	remaining: 7.65s
497:	learn: 0.9900990	total: 7.57s	remaining: 7.63s
498:	learn: 0.9900990	total: 7.59s	remaining: 7.62s
499:	learn: 0.9900990	total: 7.6s	remaining: 7.6s
500:	learn: 0.9900990	total: 7.62s	remaining: 7.59s
501:	learn: 0.9900990	total: 7.63s	remaining: 7.57s
502:	learn: 0.9900990	total: 7.65s	remaining: 7.56s
503:	learn: 0.9900990	total: 7.66s	remaining: 7.54s
504:	learn: 0.9900990	total: 7.68s	remaining: 7.53s
505:	learn: 0.9900990	total: 7.69s	remaining: 7.51s
506:	learn: 0.9900990	total: 7.71s	remaining: 7.5s
507:	learn: 0.9900990	total: 7.72s	remaining: 7.48s
508:	learn: 0.9900990	total: 7.74s	remaining: 7.46s
509:	learn: 0.9900990	total: 7.76s	remaining: 7.45s
510:	learn: 0.9900990	total: 7.77s	remaining: 7.44s
511:	learn: 0.9900990	total: 7.79s	remaining: 7.42s
512:	learn: 0.9900990	total: 7.81s	remaining: 7.41s
513:	learn: 0.9900990	total: 7.82s	remaining: 7.4s
514:	learn: 0.99

654:	learn: 0.9900990	total: 10s	remaining: 5.29s
655:	learn: 0.9900990	total: 10.1s	remaining: 5.27s
656:	learn: 0.9900990	total: 10.1s	remaining: 5.26s
657:	learn: 0.9900990	total: 10.1s	remaining: 5.24s
658:	learn: 0.9900990	total: 10.1s	remaining: 5.22s
659:	learn: 0.9900990	total: 10.1s	remaining: 5.21s
660:	learn: 0.9900990	total: 10.1s	remaining: 5.2s
661:	learn: 0.9900990	total: 10.1s	remaining: 5.18s
662:	learn: 0.9900990	total: 10.2s	remaining: 5.17s
663:	learn: 0.9900990	total: 10.2s	remaining: 5.15s
664:	learn: 0.9900990	total: 10.2s	remaining: 5.14s
665:	learn: 0.9900990	total: 10.2s	remaining: 5.12s
666:	learn: 0.9900990	total: 10.2s	remaining: 5.11s
667:	learn: 0.9900990	total: 10.2s	remaining: 5.09s
668:	learn: 0.9900990	total: 10.3s	remaining: 5.08s
669:	learn: 0.9900990	total: 10.3s	remaining: 5.06s
670:	learn: 0.9900990	total: 10.3s	remaining: 5.05s
671:	learn: 0.9900990	total: 10.3s	remaining: 5.03s
672:	learn: 0.9900990	total: 10.3s	remaining: 5.02s
673:	learn: 0.9

822:	learn: 0.9950393	total: 12.8s	remaining: 2.75s
823:	learn: 0.9950393	total: 12.8s	remaining: 2.73s
824:	learn: 0.9950393	total: 12.8s	remaining: 2.71s
825:	learn: 0.9950393	total: 12.8s	remaining: 2.7s
826:	learn: 0.9950393	total: 12.8s	remaining: 2.68s
827:	learn: 0.9950393	total: 12.8s	remaining: 2.67s
828:	learn: 0.9950393	total: 12.9s	remaining: 2.65s
829:	learn: 0.9950393	total: 12.9s	remaining: 2.64s
830:	learn: 0.9950393	total: 12.9s	remaining: 2.62s
831:	learn: 0.9950393	total: 12.9s	remaining: 2.61s
832:	learn: 0.9950393	total: 12.9s	remaining: 2.59s
833:	learn: 0.9950393	total: 12.9s	remaining: 2.58s
834:	learn: 0.9950393	total: 13s	remaining: 2.56s
835:	learn: 0.9950393	total: 13s	remaining: 2.55s
836:	learn: 0.9950393	total: 13s	remaining: 2.53s
837:	learn: 0.9950393	total: 13s	remaining: 2.52s
838:	learn: 0.9950393	total: 13s	remaining: 2.5s
839:	learn: 0.9950393	total: 13s	remaining: 2.48s
840:	learn: 0.9950393	total: 13.1s	remaining: 2.47s
841:	learn: 0.9950393	tota

988:	learn: 0.9950393	total: 15.4s	remaining: 171ms
989:	learn: 0.9950393	total: 15.4s	remaining: 156ms
990:	learn: 0.9950393	total: 15.4s	remaining: 140ms
991:	learn: 0.9950393	total: 15.4s	remaining: 125ms
992:	learn: 0.9950393	total: 15.5s	remaining: 109ms
993:	learn: 0.9950393	total: 15.5s	remaining: 93.4ms
994:	learn: 0.9950393	total: 15.5s	remaining: 77.8ms
995:	learn: 0.9950393	total: 15.5s	remaining: 62.2ms
996:	learn: 0.9950393	total: 15.5s	remaining: 46.7ms
997:	learn: 0.9950393	total: 15.5s	remaining: 31.1ms
998:	learn: 0.9950393	total: 15.5s	remaining: 15.6ms
999:	learn: 0.9950393	total: 15.6s	remaining: 0us


Делаем предсказания для каждой модели и смотрим на метрики качества

In [17]:
# Catboost 1000 деревьев
y_pred_often = clf1.predict(X[['text_str', 'title_lemmas', 'date_enc', 'year', 'season', 'day_of_week']])
print(classification_report(y, y_pred_often, target_names=dict_topic))
f1_score(y, y_pred_often, average='micro')

                   precision    recall  f1-score   support

   69-я параллель       0.65      0.70      0.68       121
           Бизнес       0.70      0.74      0.72       267
      Бывший СССР       0.86      0.90      0.88      3376
              Дом       0.84      0.83      0.84       797
    Забота о себе       0.83      0.91      0.87       114
         Из жизни       0.76      0.73      0.75      1613
   Интернет и СМИ       0.79      0.79      0.79      2333
             Крым       0.00      0.00      0.00        24
    Культпросвет        0.00      0.00      0.00        12
         Культура       0.87      0.90      0.89      2584
          Легпром       0.00      0.00      0.00         4
              Мир       0.86      0.86      0.86      7048
       Моя страна       0.83      0.84      0.84       221
  Наука и техника       0.90      0.89      0.89      2722
       Нацпроекты       0.90      0.94      0.92       170
      Путешествия       0.85      0.87      0.86       

C:\Users\sofi\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sofi\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sofi\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.859524440482787

In [18]:
# Catboost 1000 деревьев
y_pred_rarely = clf2.predict(X[['text_str', 'title_lemmas', 'date_enc', 'year', 'season', 'day_of_week']])
print(classification_report(y, y_pred_rarely, target_names=dict_topic))
f1_score(y, y_pred_rarely, average='micro')

                   precision    recall  f1-score   support

   69-я параллель       0.00      0.00      0.00       121
           Бизнес       0.00      0.00      0.00       267
      Бывший СССР       0.00      0.00      0.00      3376
              Дом       0.00      0.00      0.00       797
    Забота о себе       0.00      0.00      0.00       114
         Из жизни       0.00      0.00      0.00      1613
   Интернет и СМИ       0.00      0.00      0.00      2333
             Крым       0.00      1.00      0.00        24
    Культпросвет        0.00      1.00      0.00        12
         Культура       0.00      0.00      0.00      2584
          Легпром       0.00      1.00      0.00         4
              Мир       0.00      0.00      0.00      7048
       Моя страна       0.00      0.00      0.00       221
  Наука и техника       0.00      0.00      0.00      2722
       Нацпроекты       0.00      0.00      0.00       170
      Путешествия       0.00      0.00      0.00       

C:\Users\sofi\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sofi\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sofi\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0009636464381218531

In [61]:
# Catboost 1000 деревьев
y_pred = clf.predict(X[['text_str', 'title_lemmas', 'date_enc', 'year', 'season', 'day_of_week']])
print(classification_report(y, y_pred, target_names=dict_topic))
f1_score(y, y_pred, average='micro')

                   precision    recall  f1-score   support

   69-я параллель       0.67      0.69      0.68       121
           Бизнес       0.71      0.76      0.73       267
      Бывший СССР       0.86      0.90      0.88      3376
              Дом       0.84      0.84      0.84       797
    Забота о себе       0.84      0.93      0.88       114
         Из жизни       0.75      0.73      0.74      1613
   Интернет и СМИ       0.79      0.77      0.78      2333
             Крым       0.90      0.38      0.53        24
    Культпросвет        1.00      0.33      0.50        12
         Культура       0.87      0.90      0.89      2584
          Легпром       0.00      0.00      0.00         4
              Мир       0.85      0.86      0.86      7048
       Моя страна       0.83      0.84      0.84       221
  Наука и техника       0.90      0.89      0.90      2722
       Нацпроекты       0.89      0.93      0.91       170
      Путешествия       0.85      0.87      0.86       

0.8591389819075381

Объединим предсказания для двух моделей:
 - объединения модели, обученной на частовстречаемых темах, и модели, обученной на редковстречаемых темах
 - объединения модели, обученной на всех темах, и модели, обученной на редковстречаемых темах

In [67]:
y_pred_often1 = clf1.predict_proba(X[['text_str', 'title_lemmas', 'date_enc', 'year', 'season', 'day_of_week']])
y_pred_rarely1 = clf2.predict_proba(X[['text_str', 'title_lemmas', 'date_enc', 'year', 'season', 'day_of_week']])
y_pred1 = clf.predict_proba(X[['text_str', 'title_lemmas', 'date_enc', 'year', 'season', 'day_of_week']])

In [65]:
a = [y_pred_often[i][0] if max(y_pred_often1[i])>max(y_pred_rarely1[i]) else y_pred_rarely[i][0] for i in range(len(X))]
a1 = [y_pred[i][0] if max(y_pred1[i])>max(y_pred_rarely1[i]) else y_pred_rarely[i][0] for i in range(len(X))]
a1[:7]

[8, 8, 23, 27, 10, 8, 27]

Метрики качества для модели, обученной на частовстречаемых темах, и модели, обученной на редковстречаемых темах

In [57]:
print(classification_report(y, a, target_names=dict_topic))
f1_score(y, a, average='micro')

                   precision    recall  f1-score   support

   69-я параллель       0.94      0.28      0.43       121
           Бизнес       0.89      0.32      0.47       267
      Бывший СССР       0.96      0.38      0.55      3376
              Дом       0.94      0.61      0.74       797
    Забота о себе       0.92      0.21      0.34       114
         Из жизни       0.89      0.27      0.42      1613
   Интернет и СМИ       0.88      0.40      0.55      2333
             Крым       0.00      1.00      0.00        24
    Культпросвет        0.01      1.00      0.01        12
         Культура       0.95      0.48      0.64      2584
          Легпром       0.01      1.00      0.01         4
              Мир       0.95      0.43      0.59      7048
       Моя страна       0.96      0.40      0.56       221
  Наука и техника       0.96      0.65      0.77      2722
       Нацпроекты       0.94      0.66      0.78       170
      Путешествия       0.97      0.45      0.62       

0.4955310896432099

Метрики качества для модели, обученной на всех темах, и модели, обученной на редковстречаемых темах

In [66]:
print(classification_report(y, a1, target_names=dict_topic))
f1_score(y, a1, average='micro')

                   precision    recall  f1-score   support

   69-я параллель       0.90      0.29      0.44       121
           Бизнес       0.88      0.34      0.49       267
      Бывший СССР       0.96      0.38      0.55      3376
              Дом       0.95      0.60      0.74       797
    Забота о себе       0.97      0.25      0.40       114
         Из жизни       0.89      0.29      0.44      1613
   Интернет и СМИ       0.89      0.41      0.56      2333
             Крым       0.00      1.00      0.00        24
    Культпросвет        0.01      1.00      0.01        12
         Культура       0.95      0.48      0.64      2584
          Легпром       0.01      1.00      0.01         4
              Мир       0.95      0.43      0.59      7048
       Моя страна       0.92      0.42      0.57       221
  Наука и техника       0.96      0.65      0.77      2722
       Нацпроекты       0.93      0.66      0.77       170
      Путешествия       0.97      0.43      0.60       

0.4953142691946325

**Выводы**:
 1. f1-score нереально сильно упал (стал в районе 0.5) 
 2. Данный подход не помог
 3. Возможно он не подходит для модели CatBoost